In [ ]:
import pycaret
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import inchi

from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

from rdkit import DataStructs

Tanimoto similarity

Validate the 7 triple positive hits: are they similar to the known Mcl-1/Bcl-2 inhibitors?

In [ ]:
#Load the Mcl-1 and Bc-2 datasets
mcl_raw = pd.read_csv('../data_preparation/inhibitors/mcl/mcl_molecules.csv', index_col=0) #865 compounds
bcl_raw = pd.read_csv('../data_preparation/inhibitors/bcl/bcl_molecules.csv', index_col=0) #428 compounds

#Filter for positive molecules: inhibitors
mcl = mcl_raw[mcl_raw['Class'] == 'Inhibitor'] #298 compounds
mcl = mcl.reset_index(drop=True)
mcl = mcl.rename(columns={'SMILES': 'papyrus_SMILES'})

bcl = bcl_raw[bcl_raw['Class'] == 'Inhibitor'] #125 compounds
bcl = bcl.reset_index(drop=True)
bcl = bcl.rename(columns={'SMILES': 'papyrus_SMILES'})

In [ ]:
#Load the seven triple positive hits
hits = pd.read_csv('triple_pos.csv')

columns_to_keep = ['papyrus_SMILES', 'composite_score']
hits = hits[columns_to_keep]

In [ ]:
#Generate SMILES list
def generate_smiles(df):
    smiles_list = df['papyrus_SMILES'].tolist()

    return smiles_list

In [ ]:
#Generate molecules from SMILES

def generate_mols(smiles_list):
    mols = []
    smiles_to_fix = []

    for smiles in smiles_list:
        try:
            mol = Chem.MolFromSmiles(smiles)
            mols.append(mol)

        except Exception as e:
            print(f"Error with SMILES string {smiles}")
            continue 


    #check for None Mol objects
    for i, mol in enumerate(mols):
        if mol is None:
            print(f"Error with SMILES string at index {i}: {smiles_list[i]}")
            smiles_to_fix.append(i)
            continue

    return mols

In [ ]:
#Add hydrogens to molecules

def add_hydrogens(mols):

    mols_H = []

    for mol in mols:
        mol_H = Chem.AddHs(mol)
        mols_H.append(mol_H)

    return mols_H

In [ ]:
#Create Morgan Fingerprints
def morgan_fp(mols_H):

    fp = []

    print(f'Calculating Morgan fingerprints...')

    radius = 2 #define radius
    nBits = 2048   #define number of bits

    for mol in mols_H:
        new_data = Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius, nBits)
        fp.append(new_data)

    return fp

In [ ]:
#Create Morgan FPs for known Mcl-1 inhibitors
df = mcl

smiles_list= generate_smiles(df)
mols = generate_mols(smiles_list)
mols_H = add_hydrogens(mols)

mcl_morgan= morgan_fp(mols_H)



In [ ]:
#Create Morgan FPs for known Bcl-2 inhibitors
df = bcl

smiles_list= generate_smiles(df)
mols = generate_mols(smiles_list)
mols_H = add_hydrogens(mols)

bcl_morgan= morgan_fp(mols_H)



In [ ]:
#Create Morgan FPs for triple positive hits
df = hits

smiles_list= generate_smiles(df)
mols = generate_mols(smiles_list)
mols_H = add_hydrogens(mols)

hits_morgan= morgan_fp(mols_H)


In [ ]:
#Generate similarity matrix: 7 triple hits compared to known Mcl-1 inhibitors

mcl_similarity_matrix = []

#Iterate through hits compounds
for hit_fp in hits_morgan:
    mcl_hit_similarities = []

    #Iterate through Mcl-1 inhibitors
    for mcl_fp in mcl_morgan:

        #Calculate similarity
        similarity = DataStructs.TanimotoSimilarity(hit_fp, mcl_fp)
        mcl_hit_similarities.append(similarity)

    mcl_similarity_matrix.append(mcl_hit_similarities)

print("Mcl-1 Similarity matrix:")
for row in mcl_similarity_matrix:
    print(row)

In [ ]:
print('Mcl-1 average, maximum and min similarity:')

for i, row in enumerate(mcl_similarity_matrix):
    mcl_avg_similarity = sum(row) / len(row)  #Calculate average
    mcl_max_similarity = max(row)     # Find max
    mcl_min_similarity = min(row)     #Find min
    
    print(f"Row {i + 1}: Average = {mcl_avg_similarity}, Maximum = {mcl_max_similarity}, Minimum = {mcl_min_similarity}")

In [ ]:
#Generate similarity matrix: 7 triple hits compared to known Bcl-2 inhibitors

bcl_similarity_matrix = []

#Iterate through hits compounds
for hit_fp in hits_morgan:
    bcl_hit_similarities = []
    
    #Iterate through known Bcl-2 inhibitors
    for bcl_fp in bcl_morgan:

        #Calculate similarity
        similarity = DataStructs.TanimotoSimilarity(hit_fp, bcl_fp)
        bcl_hit_similarities.append(similarity)

    bcl_similarity_matrix.append(bcl_hit_similarities)

print("Bcl-1 Similarity matrix:")
for row in bcl_similarity_matrix:
    print(row)

In [ ]:
print('Bcl-2 average, max and min similarity:')
for i, row in enumerate(bcl_similarity_matrix):
    bcl_avg_similarity = sum(row) / len(row)  #Calculate average
    bcl_max_similarity = max(row)        #Find max
    bcl_min_similarity = min(row)        #Find min
    print(f"Row {i + 1}: Average = {bcl_avg_similarity}, Maximum = {bcl_max_similarity}, Minimum = {bcl_min_similarity}")

Mcl-to-Mcl Similarity

In [ ]:
#Generate similarity matrix: known Mcl-1 inhibitors compared to Mcl-1 inhibitors

mcl_mcl_similarity_matrix = []

for mcl1_fp in mcl_morgan:
    mcl_mcl_similarities = []

    for mcl2_fp in mcl_morgan:
        similarity = DataStructs.TanimotoSimilarity(mcl1_fp, mcl2_fp)
        mcl_mcl_similarities.append(similarity)

    mcl_mcl_similarity_matrix.append(mcl_mcl_similarities)

In [ ]:
# Initialize variables to count occurrences of "1" and store maximum value
max_value_total = float('-inf')
total_sum_total = 0
count_non_ones_total = 0

# Initialize variables to track min and max average values over rows
min_average = float('inf')
max_average = float('-inf')

# Iterate through each row of the similarity matrix
for row in mcl_mcl_similarity_matrix:

    # Initialize variables for each row
    max_value_row = float('-inf')
    total_sum_row = 0
    count_non_ones_row = 0
    
    # Iterate through each element in the row
    for value in row:

        # Check if the value is equal to 1
        if value != 1:

            # Add the non-"1" value to the total sum for the row
            total_sum_row += value

            # Increment the count of non-"1" values for the row
            count_non_ones_row += 1

    # Calculate the average for the row without 1s
    average_row = total_sum_row / count_non_ones_row if count_non_ones_row > 0 else 0
    
    # Update min and max average values over rows
    min_average = min(min_average, average_row)
    max_average = max(max_average, average_row)

# Print the min and max average values over rows
print(f"Mcl-Mcl Minimum Average over Rows: {min_average}")
print(f"Mcl-Mcl Maximum Average over Rows: {max_average}")

Bcl-to-Bcl similarity

In [ ]:
#Generate similarity matrix: known Bcl-2 inhibitors compared to Bcl-2 inhibitors

bcl_bcl_similarity_matrix = []

#Iterate throguh inhibitors 
for bcl1_fp in bcl_morgan:
    bcl_bcl_similarities = []

    #Iterate through inhibitors
    for bcl2_fp in bcl_morgan:

        #Calculate similarity
        similarity = DataStructs.TanimotoSimilarity(bcl1_fp, bcl2_fp)
        bcl_bcl_similarities.append(similarity)

    bcl_bcl_similarity_matrix.append(bcl_bcl_similarities)




In [ ]:
# Initialize variables to count occurrences of "1" and store maximum value
max_value_total = float('-inf')
total_sum_total = 0
count_non_ones_total = 0

# Initialize variables to track min and max average values over rows
min_average = float('inf')
max_average = float('-inf')

# Iterate through each row of the similarity matrix
for row in bcl_bcl_similarity_matrix:

    # Initialize variables for each row
    max_value_row = float('-inf')
    total_sum_row = 0
    count_non_ones_row = 0
    
    # Iterate through each element in the row
    for value in row:

        # Check if the value is equal to 1
        if value != 1:

            # Add the non-"1" value to the total sum for the row
            total_sum_row += value

            # Increment the count of non-"1" values for the row
            count_non_ones_row += 1

    # Calculate the average for the row without 1s
    average_row = total_sum_row / count_non_ones_row if count_non_ones_row > 0 else 0
    
    # Update min and max average values over rows
    min_average = min(min_average, average_row)
    max_average = max(max_average, average_row)

# Print the min and max average values over rows
print(f"Bcl-Bcl Minimum Average over Rows: {min_average}")
print(f"Bcl-Bcl Maximum Average over Rows: {max_average}")